In [152]:
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import csr_matrix, hstack
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [2]:
PATH_TO_DATA = ('data')
train_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_sessions.csv'), index_col='session_id')
test_df = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_sessions.csv'), index_col='session_id')

In [4]:
train_df

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
1,718,2014-02-20 10:02:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,890,2014-02-22 11:19:50,941.0,2014-02-22 11:19:50,3847.0,2014-02-22 11:19:51,941.0,2014-02-22 11:19:51,942.0,2014-02-22 11:19:51,...,2014-02-22 11:19:51,3847.0,2014-02-22 11:19:52,3846.0,2014-02-22 11:19:52,1516.0,2014-02-22 11:20:15,1518.0,2014-02-22 11:20:16,0
3,14769,2013-12-16 16:40:17,39.0,2013-12-16 16:40:18,14768.0,2013-12-16 16:40:19,14769.0,2013-12-16 16:40:19,37.0,2013-12-16 16:40:19,...,2013-12-16 16:40:19,14768.0,2013-12-16 16:40:20,14768.0,2013-12-16 16:40:21,14768.0,2013-12-16 16:40:22,14768.0,2013-12-16 16:40:24,0
4,782,2014-03-28 10:52:12,782.0,2014-03-28 10:52:42,782.0,2014-03-28 10:53:12,782.0,2014-03-28 10:53:42,782.0,2014-03-28 10:54:12,...,2014-03-28 10:54:42,782.0,2014-03-28 10:55:12,782.0,2014-03-28 10:55:42,782.0,2014-03-28 10:56:12,782.0,2014-03-28 10:56:42,0
5,22,2014-02-28 10:53:05,177.0,2014-02-28 10:55:22,175.0,2014-02-28 10:55:22,178.0,2014-02-28 10:55:23,177.0,2014-02-28 10:55:23,...,2014-02-28 10:55:59,175.0,2014-02-28 10:55:59,177.0,2014-02-28 10:55:59,177.0,2014-02-28 10:57:06,178.0,2014-02-28 10:57:11,0
6,570,2014-03-18 15:18:31,21.0,2014-03-18 15:18:39,570.0,2014-03-18 15:23:02,21.0,2014-03-18 15:23:43,21.0,2014-03-18 15:29:57,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,803,2014-02-13 16:45:35,23.0,2014-02-13 16:45:35,5956.0,2014-02-13 16:45:35,17513.0,2014-02-13 16:45:35,37.0,2014-02-13 16:46:05,...,2014-02-13 16:47:14,803.0,2014-02-13 16:47:14,17514.0,2014-02-13 16:47:15,17514.0,2014-02-13 16:47:16,17514.0,2014-02-13 16:47:17,0
8,22,2013-04-12 10:27:26,21.0,2013-04-12 10:27:26,29.0,2013-04-12 10:27:28,5041.0,2013-04-12 10:27:29,14422.0,2013-04-12 10:27:29,...,2013-04-12 10:27:29,21.0,2013-04-12 10:27:29,5041.0,2013-04-12 10:27:31,14421.0,2013-04-12 10:27:31,14421.0,2013-04-12 10:27:32,0
9,668,2014-03-17 16:23:08,940.0,2014-03-17 16:23:35,942.0,2014-03-17 16:23:35,941.0,2014-03-17 16:23:35,941.0,2014-03-17 16:23:36,...,2014-03-17 16:23:36,940.0,2014-03-17 16:23:36,23.0,2014-03-17 16:23:52,21.0,2014-03-17 16:23:52,22.0,2014-03-17 16:23:53,0


In [3]:
y = train_df['target']

In [5]:
times = ['time%s' % i for i in range(1, 11)]

In [7]:
train_df[times] = train_df[times].apply(pd.to_datetime)

In [9]:
test_df[times] = test_df[times].apply(pd.to_datetime)

In [10]:
train_df = train_df.sort_values(by = 'time1')

In [11]:
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [12]:
sites = ['site%s' % i for i in range(1, 11)]

In [13]:
train_df[sites] = train_df[sites].fillna(0).astype('int')

In [14]:
test_df[sites] = test_df[sites].fillna(0).astype('int')

In [17]:
with open(r"data/site_dic.pkl", "rb") as input_file:
    site_dict = pickle.load(input_file)

In [18]:
sites_dict_df = pd.DataFrame(list(site_dict.keys()),
                            index = list(site_dict.values()),
                            columns = ['site'])

In [19]:
print(u'total sites:', sites_dict_df.shape[0])

total sites: 48371


In [20]:
sites_dict_df.head()

,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [21]:
y_train = train_df['target']

In [22]:
full_df = pd.concat([train_df.drop('target', axis = 1), test_df])

In [29]:
idx_split = train_df.shape[0]

In [24]:
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


In [258]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,3) ,max_features=100000, token_pattern=r'(?u)\b\w+\b')

In [254]:
full_sites['inline'] = full_sites.to_string(index = False, header = False).split('\n')

In [262]:
full_sites_sparse = tfidf_vectorizer.fit_transform(full_sites['inline'])

In [25]:
from scipy.sparse import csr_matrix

In [26]:
csr_matrix?

In [27]:
full_sites.values.flatten()

array([  56,   55,    0, ..., 1098, 1098, 1098])

In [28]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()

# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]

In [263]:
X_train_sparse = full_sites_sparse[:idx_split]
X_test_sparse = full_sites_sparse[idx_split:]

In [264]:
X_train_sparse.shape, y_train.shape

((253561, 100000), (253561,))

In [265]:
X_test_sparse.shape

(82797, 100000)

In [141]:
def get_auc_lr_valid(X, y, C = 1.0, ratio = 0.9, seed = 17):
    train_len = int(ratio * X.shape[0])
    X_train = X[:train_len, :]
    X_valid = X[train_len:, :]
    y_train = y[:train_len]
    y_valid = y[train_len:]
    
    logit = LogisticRegression(C = C, n_jobs = -1, random_state = seed)
    
    logit.fit(X_train, y_train)
    
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    return roc_auc_score(y_valid, valid_pred)

In [35]:
get_auc_lr_valid(X_train_sparse, y_train)

0.9195289734309581

In [37]:
def write_to_submission_file(predicted_labels, out_file,
                            target = 'target', index_labes = "session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                               index = np.arange(1, predicted_labels.shape[0] + 1),
                               columns = [target])
    predicted_df.to_csv(out_file, index_label = index_label)

In [38]:
%%time
logit = LogisticRegression(n_jobs=-1, random_state=17)
logit.fit(X_train_sparse, y_train)

CPU times: user 3.7 s, sys: 28.7 ms, total: 3.73 s
Wall time: 3.73 s


In [39]:
test_pred = logit.predict_proba(X_test_sparse)[:, 1]

In [40]:
test_pred.shape

(82797,)

In [51]:
pd.Series(test_pred, index = range(1, test_pred.shape[0] + 1),
         name = 'target').to_csv('benchmark1.csv', header = True, index_label = 'session_id')

In [52]:
!head benchmark1.csv

session_id,target
1,0.0022099552636363606
2,4.808795918257945e-09
3,1.8729296476310883e-08
4,2.353990648464104e-08
5,3.131124341228164e-05
6,0.00021845651476912005
7,0.0005478776444933022
8,0.00013228235242444572
9,0.0007952123906636395


In [54]:
time = ['time%d' % i for i in range(1, 11)]

In [65]:
new_feat_train = pd.DataFrame(index = train_df.index)
new_feat_test = pd.DataFrame(index = test_df.index)

In [237]:
new_feat_train['month'] = train_df['time1'].apply(lambda ts: ts.month)
new_feat_test['month'] = test_df['time1'].apply(lambda ts: ts.month)

In [67]:
new_feat_train.head()

,year_month
session_id,
21669,201301
54843,201301
77292,201301
114021,201301
146670,201301


In [238]:
scaler = StandardScaler()
scaler.fit(new_feat_train['month'].values.reshape(-1, 1))

new_feat_train['month_scaled'] = scaler.transform(new_feat_train['month'].values.reshape(-1, 1))
new_feat_test['month_scaled'] = scaler.transform(new_feat_test['month'].values.reshape(-1, 1))

In [73]:
new_feat_train.head()

,year_month,year_month_scaled
session_id,,
21669,201301,-1.744405
54843,201301,-1.744405
77292,201301,-1.744405
114021,201301,-1.744405
146670,201301,-1.744405


In [266]:
X_train_sparse_new = csr_matrix(hstack([X_train_sparse,
                                        new_feat_train['month_scaled'].values.reshape(-1,1)]))
X_test_sparse_new = csr_matrix(hstack([X_test_sparse,
                                        new_feat_test['month_scaled'].values.reshape(-1,1)]))

In [82]:
%%time

get_auc_lr_valid(X_train_sparse_new, y_train)

CPU times: user 2.91 s, sys: 37.4 ms, total: 2.95 s
Wall time: 2.95 s


0.9196929090450192

In [80]:
X_train_sparse_new.shape

(253561, 48372)

In [227]:
new_feat_train['start_hour'] = train_df['time1'].apply(lambda ts: ts.hour)
new_feat_test['start_hour'] = test_df['time1'].apply(lambda ts: ts.hour)

In [228]:
scaler = StandardScaler()
scaler.fit(new_feat_train['start_hour'].values.reshape(-1, 1))

new_feat_train['start_hour_scaled'] = scaler.transform(new_feat_train['start_hour'].values.reshape(-1, 1))
new_feat_test['start_hour_scaled'] = scaler.transform(new_feat_test['start_hour'].values.reshape(-1, 1))

In [94]:
new_feat_train.head()

,year_month,year_month_scaled,start_hour,start_hour_scaled,morning
session_id,,,,,
21669,201301,-1.744405,8,-1.357366,1
54843,201301,-1.744405,8,-1.357366,1
77292,201301,-1.744405,8,-1.357366,1
114021,201301,-1.744405,8,-1.357366,1
146670,201301,-1.744405,8,-1.357366,1


In [229]:
X_train_sparse_new1 = csr_matrix(hstack([X_train_sparse_new,
                                        new_feat_train['start_hour_scaled'].values.reshape(-1,1)]))

In [230]:
X_test_sparse_new1 = csr_matrix(hstack([X_test_sparse_new,
                                        new_feat_test['start_hour_scaled'].values.reshape(-1,1)]))

In [116]:
%%time

get_auc_lr_valid(X_train_sparse_new, y_train)

CPU times: user 2.95 s, sys: 37.5 ms, total: 2.98 s
Wall time: 3 s


0.9196929090450192

In [115]:
%%time
get_auc_lr_valid(X_train_sparse_new1, y_train)

CPU times: user 3.14 s, sys: 49.7 ms, total: 3.19 s
Wall time: 3.2 s


0.9579182863988388

In [183]:
# new_feat_train['morning'] = train_df['time1'].apply(lambda x: 1 if x.hour > 8 and x.hour <= 11 else 0)
# new_feat_test['morning'] = test_df['time1'].apply(lambda x: 1 if x.hour > 8 and x.hour <= 11 else 0)

In [184]:
new_feat_train.sample(5)

,year_month,year_month_scaled,start_hour,start_hour_scaled,morning,day,evening,night,dow,is_weekend,is_sunday,is_begin_month
session_id,,,,,,,,,,,,
53320,201404,0.681626,15,0.858234,0,1,0,0,0,0,0,1
164405,201304,-1.673744,13,0.225205,0,1,0,0,4,0,0,1
227861,201311,-1.508868,15,0.858234,0,1,0,0,0,0,0,0
114766,201312,-1.485314,22,3.073835,0,0,1,0,6,1,1,1
52797,201305,-1.650190,17,1.491263,0,1,0,0,6,1,1,1


In [186]:
X_train_sparse_new2 = csr_matrix(hstack([X_train_sparse_new1,
                                        new_feat_train['morning'].values.reshape(-1,1)]))
X_test_sparse_new2 = csr_matrix(hstack([X_test_sparse_new1,
                                        new_feat_test['morning'].values.reshape(-1,1)]))

In [187]:
%%time
get_auc_lr_valid(X_train_sparse_new2, y_train)

CPU times: user 3 s, sys: 36.3 ms, total: 3.04 s
Wall time: 3.04 s


0.9592743026245397

In [219]:
new_feat_train['day1'] = train_df['time1'].apply(lambda x: 1 if x.hour >= 12 and x.hour < 14 else 0)
new_feat_test['day1'] = test_df['time1'].apply(lambda x: 1 if x.hour >= 12 and x.hour < 14 else 0)

new_feat_train['morning'] = train_df['time1'].apply(lambda x: 1 if x.hour > 8 and x.hour <= 11 else 0)
new_feat_test['morning'] = test_df['time1'].apply(lambda x: 1 if x.hour > 8 and x.hour <= 11 else 0)

new_feat_train['day2'] = train_df['time1'].apply(lambda x: 1 if x.hour > 15 and x.hour <= 18 else 0)
new_feat_test['day2'] = test_df['time1'].apply(lambda x: 1 if x.hour > 15 and x.hour <= 18 else 0)

new_feat_train['evening'] = train_df['time1'].apply(lambda x: 1 if x.hour > 18 else 0)
new_feat_test['evening'] = test_df['time1'].apply(lambda x: 1 if x.hour > 18 else 0)

new_feat_train['night'] = train_df['time1'].apply(lambda x: 1 if x.hour <= 8 else 0)
new_feat_test['night'] = test_df['time1'].apply(lambda x: 1 if x.hour <= 8 else 0)

In [280]:
X_train_sparse_new3 = csr_matrix(hstack([X_train_sparse_new,
                                        new_feat_train['day1'].values.reshape(-1,1),
                                         new_feat_train['morning'].values.reshape(-1,1),
                                         new_feat_train['day2'].values.reshape(-1,1),
                                        new_feat_train['evening'].values.reshape(-1,1),
                                        new_feat_train['night'].values.reshape(-1,1),
                                        new_feat_train['is_sunday'].values.reshape(-1,1),
                                        new_feat_train['is_begin_month'].values.reshape(-1,1)
#                                          new_feat_train['dow'].values.reshape(-1,1)
                                        ]))
X_test_sparse_new3 = csr_matrix(hstack([X_test_sparse_new,
                                        new_feat_test['day1'].values.reshape(-1,1),
                                        new_feat_test['morning'].values.reshape(-1,1),
                                        new_feat_test['day2'].values.reshape(-1,1),
                                        new_feat_test['evening'].values.reshape(-1,1),
                                        new_feat_test['night'].values.reshape(-1,1),
                                       new_feat_test['is_sunday'].values.reshape(-1,1),
#                                         new_feat_test['dow'].values.reshape(-1,1),
                                       new_feat_test['is_begin_month'].values.reshape(-1,1)
                                       ]))

In [194]:
%%time
get_auc_lr_valid(X_train_sparse_new3, y_train)

CPU times: user 2.78 s, sys: 33.6 ms, total: 2.82 s
Wall time: 2.82 s


0.9759279570905323

In [ ]:
0.9629554572143143

In [190]:
new_feat_train['dow'] = train_df['time1'].apply(lambda x: x.date().weekday())
new_feat_train['is_weekend'] = train_df['time1'].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)

new_feat_test['dow'] = test_df['time1'].apply(lambda x: x.date().weekday())
new_feat_test['is_weekend'] = test_df['time1'].apply(lambda x: 1 if x.date().weekday() in (5, 6) else 0)

In [191]:
new_feat_train['is_sunday'] = train_df['time1'].apply(lambda x: 1 if x.date().weekday() == 6 else 0)
new_feat_test['is_sunday'] = test_df['time1'].apply(lambda x: 1 if x.date().weekday() == 6 else 0)

In [274]:
new_feat_train['is_begin_month'] = train_df['time1'].apply(lambda x: 1 if x.day < 15 else 0)
new_feat_test['is_begin_month'] = test_df['time1'].apply(lambda x: 1 if x.day < 15 else 0)

In [245]:
params = {
        'penalty': ['l2', 'l1'],
         'C': np.logspace(-3, 1, 10),
         'class_weight': ['balanced', None]
}

In [246]:
logit = LogisticRegression(n_jobs=-1, random_state=17)

In [164]:
%%time
logit.fit(X_train_sparse_new3, y_train)

CPU times: user 7.54 s, sys: 27.4 ms, total: 7.57 s
Wall time: 7.61 s


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=-1, penalty='l2', random_state=17,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [196]:
sp = TimeSeriesSplit(3)

In [247]:
res = GridSearchCV(logit, params, n_jobs = -1, verbose = 1, scoring = 'roc_auc', cv=sp)

In [281]:
%%time
res.fit(X_train_sparse_new3, y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   22.5s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  3.2min finished


CPU times: user 21 s, sys: 582 ms, total: 21.6 s
Wall time: 3min 21s


GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=3),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=-1,
          penalty='l2', random_state=17, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'penalty': ['l2', 'l1'], 'C': array([1.00000e-03, 2.78256e-03, 7.74264e-03, 2.15443e-02, 5.99484e-02,
       1.66810e-01, 4.64159e-01, 1.29155e+00, 3.59381e+00, 1.00000e+01]), 'class_weight': ['balanced', None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=1)

In [282]:
res.best_params_

{'C': 10.0, 'class_weight': None, 'penalty': 'l1'}

In [283]:

res.best_score_

0.9488015648113525

0.9494626592986343 baseline <br>
0.9494920763137062 {'C': 1.2915496650148828, 'class_weight': None, 'penalty': 'l2'} <br>
tfidf: 0.9545406078001493 ({'C': 10.0, 'class_weight': None, 'penalty': 'l2'} <br>
tfidf: 0.954929266993279

In [249]:
res.best_params_

{'C': 1.2915496650148828, 'class_weight': None, 'penalty': 'l2'}

In [250]:
res.best_score_

0.9494920763137062

In [242]:
res.best_score_

0.9494626592986343

# Vectorizing

In [273]:
test_pred = res.predict_pr11oba(X_test_sparse_new3)[:, 1]
pd.Series(test_pred, index = range(1, test_pred.shape[0] + 1),
         name = 'target').to_csv('benchmark5.csv', header = True, index_label = 'session_id')

In [ ]:
alise_hours = defaultdict(int)
alise_hours[16] = 879
alise_hours[17] = 618
alise_hours[12] = 340
alise_hours[13] = 196
alise_hours[18] = 179
alise_hours[15] =  40
alise_hours[9]  =  38
alise_hours[14] =   4
alise_hours[11] =   3

alise_weekday = defaultdict(int)
alise_weekday[0]= 876
alise_weekday[1]= 508
alise_weekday[3]= 487
alise_weekday[4]= 346
alise_weekday[5]=  41
alise_weekday[2]=  38
alise_weekday[6]=   1